In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv
              
import requests
              
import json
from pandas.io.json import json_normalize




In [4]:
load_dotenv()
client_id = os.getenv('foursquare_id')
client_secret = os.getenv('foursquare_secret')
version = '20200508'


In [11]:
dallas_df = pd.read_csv('./Data/dfw_demographics.csv')

In [12]:
dallas_df.head()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,75001,Addison,32.96,-96.84,12414,57633.0,258200.0,0.092103,0.366306,0.038184,0.332532,0.144217,0.023251,0.003407
1,75002,Allen,33.08,-96.61,63140,91632.0,186100.0,0.052425,0.416282,0.085043,0.310966,0.109292,0.015340,0.010652
2,75006,Carrollton,32.95,-96.89,46364,56985.0,146800.0,0.252946,0.383821,0.066069,0.209083,0.075295,0.010197,0.002590
3,75007,Carrollton,33.00,-96.90,51624,78934.0,169400.0,0.101158,0.426112,0.081565,0.288381,0.075876,0.014833,0.012075
4,75009,Celina,33.30,-96.80,8785,87581.0,190500.0,0.073749,0.525198,0.106760,0.215979,0.069008,0.006146,0.003161


In [32]:
dallas_df.tail()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
265,76487,Poolville,33.000,-97.90,2507,54872.0,132800.0,0.151007,0.605369,0.070470,0.141611,0.031544,0.000000,0.0
266,76490,Whitt,32.957,-98.02,91,107344.0,32500.0,0.172840,0.753086,0.074074,0.000000,0.000000,0.000000,0.0
267,76623,Avalon,32.300,-96.80,242,41250.0,80700.0,0.410526,0.536842,0.000000,0.052632,0.000000,0.000000,0.0
268,76651,Italy,32.200,-96.90,3300,50457.0,90300.0,0.224719,0.588764,0.048989,0.106067,0.029663,0.001798,0.0
269,76670,Milford,32.100,-97.00,1377,49667.0,86700.0,0.192175,0.660529,0.036824,0.100115,0.005754,0.004603,0.0


In [26]:
def get_nearby_venues(names, latitudes, longitudes, radius = 1000, limit = 500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id,
            client_secret,
            version,
            lat,
            lng,
            radius,
            limit)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)       

In [27]:
dallas_venues = get_nearby_venues(names = dallas_df['Name'],
                                   latitudes = dallas_df['lat'],
                                   longitudes = dallas_df['lng']
                                  )

In [29]:
dallas_venues.shape

(4215, 7)

In [30]:
dallas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,32.96,-96.84,La Spiga,32.958278,-96.837417,Bakery
1,Addison,32.96,-96.84,Cindi's N.Y. Delicatessen,32.961864,-96.838873,Diner
2,Addison,32.96,-96.84,Elite Cigar Cafe,32.954967,-96.837875,Smoke Shop
3,Addison,32.96,-96.84,Best Thai Signature,32.954790,-96.841320,Thai Restaurant
4,Addison,32.96,-96.84,Anime Pop,32.954681,-96.841196,Hobby Shop


In [31]:
dallas_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4210,Italy,32.2,-96.9,Italy Inn,32.197116,-96.899881,Hotel
4211,Italy,32.2,-96.9,Grandy's,32.196979,-96.899930,Diner
4212,Italy,32.2,-96.9,Monolithic Domes Bruco,32.197193,-96.898630,Scenic Lookout
4213,Italy,32.2,-96.9,Taco Bell,32.196429,-96.900353,Fast Food Restaurant
4214,Italy,32.2,-96.9,Shell,32.195247,-96.897644,Gas Station


In [33]:
dallas_venues.to_csv('./Data/dallas_venues.csv', index = False)

In [34]:
houston_df = pd.read_csv('./Data/houston_demographics.csv')

In [35]:
houston_df.head()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,77002,"Downtown, Midtown",29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687
1,77003,East Downtown,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715
2,77004,"South Central, Greater Third Ward",29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400
3,77005,West University Place,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904
4,77006,"Montrose, Hyde Park",29.74,-95.39,19664,67458.0,327300.0,0.033678,0.264526,0.030957,0.339017,0.172018,0.101215,0.058589


In [36]:
houston_df.tail()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
229,77665,Winnie,29.80,-94.30,6446,50149.0,116400.0,0.249186,0.575733,0.035831,0.100434,0.030130,0.005157,0.003529
230,78931,Bleiblerville,30.03,-96.44,323,67917.0,116400.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
231,78933,Cat Spring,29.80,-96.40,1433,59974.0,225700.0,0.049579,0.610851,0.037418,0.211413,0.061740,0.020580,0.008419
232,78944,Industry,30.00,-96.50,562,47083.0,154900.0,0.230461,0.561122,0.068136,0.036072,0.102204,0.002004,0.000000
233,78950,New Ulm,29.90,-96.50,2038,49261.0,170900.0,0.086690,0.559408,0.134115,0.150943,0.056604,0.012239,0.000000


In [37]:
houston_venues = get_nearby_venues(names = houston_df['Name'],
                                   latitudes = houston_df['lat'],
                                   longitudes = houston_df['lng']
                                  )

In [38]:
houston_venues.shape

(3714, 7)

In [39]:
houston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Downtown, Midtown",29.75,-95.37,Tellepsen Family Downtown YMCA,29.752068,-95.370679,Gym
1,"Downtown, Midtown",29.75,-95.37,Your Pie,29.751770,-95.368600,Pizza Place
2,"Downtown, Midtown",29.75,-95.37,Louisiana Famous Fried Chicken,29.748251,-95.370186,Fried Chicken Joint
3,"Downtown, Midtown",29.75,-95.37,Savarese Fight Fit Boxing Gym,29.750333,-95.365737,Gym
4,"Downtown, Midtown",29.75,-95.37,Which Wich? Superior Sandwiches,29.750875,-95.375399,Sandwich Place


In [40]:
houston_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3709,Stowell,29.7872,-94.3833,Papa's Place,29.794704,-94.383951,Seafood Restaurant
3710,Stowell,29.7872,-94.3833,Tony's Barbecue & Steak House,29.794752,-94.384161,BBQ Joint
3711,Stowell,29.7872,-94.3833,Betsy Elizabeth Photography,29.791513,-94.392021,Photography Studio
3712,Industry,30.0000,-96.5000,Industry Volunteer Fire Department Dance Halll,30.001581,-96.493831,Concert Hall
3713,New Ulm,29.9000,-96.5000,Fireman's Park,29.897545,-96.503922,Park


In [41]:
houston_venues.to_csv('./Data/houston_venues.csv', index = False)

In [3]:
dallas_venues = pd.read_csv('./Data/dallas_venues.csv')

In [8]:
dallas_venues['Neighborhood'].value_counts()

Fort Worth                                      567
Arlington                                       280
Irving                                          129
Denton                                          112
Mesquite                                        106
Turtle Creek, Oaklawn                           100
Historic District                               100
Richardson                                      100
Downtown                                        100
Plano                                           100
Deep Ellum                                      100
Garland                                          88
Lewisville                                       84
Hurst                                            81
Flower Mound                                     77
Grapevine                                        74
Carrollton                                       73
City Place, Uptown                               70
Addison                                          70
Old East    

In [5]:
print('There are {} uniques categories.'.format(len(dallas_venues['Venue Category'].unique())))

There are 325 uniques categories.


In [6]:
dallas_onehot = pd.get_dummies(dallas_venues[['Venue Category']], prefix="", prefix_sep="")

dallas_onehot['Neighborhood'] = dallas_venues['Neighborhood'] 

fixed_columns = [dallas_onehot.columns[-1]] + list(dallas_onehot.columns[:-1])
dallas_onehot = dallas_onehot[fixed_columns]

dallas_onehot.head()


,Zoo,ATM,Accessories Store,Adult Boutique,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Bookstore,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piano Bar,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,RV Park,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Street Art,Super

In [7]:
dallas_onehot.shape

(4215, 325)